In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
# Load the data from the 'Loan_Data.csv' file into a DataFrame named df
df = pd.read_csv('Loan_Data.csv')
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [3]:
df.shape

(10000, 8)

In [4]:
df.dtypes

customer_id                   int64
credit_lines_outstanding      int64
loan_amt_outstanding        float64
total_debt_outstanding      float64
income                      float64
years_employed                int64
fico_score                    int64
default                       int64
dtype: object

Le DataFrame contient 10k observations et 8 variables, toutes numériques.

In [5]:
# Find columns with NaN values
# Count NaN values for each column
nan_counts = df.isna().sum()

# Filter and print only the columns with NaN values and their counts
nan_columns_counts = nan_counts[nan_counts > 0]
nan_columns_counts

Series([], dtype: int64)

La DataFrame ne contient aucun NaN.

In [9]:
def standard_columns(column):
    return (column - column.mean()) / column.std() 

# Standardize the floats variables
df[df.select_dtypes(["float"]).columns] = df[
    df.select_dtypes(["float"]).columns 
].apply(standard_columns)
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,0.747058,-0.724812,0.398535,5,605,0
1,7442532,5,-1.548297,-0.073963,-2.161768,2,572,1
2,2256073,0,-0.560481,-1.009645,-0.207909,4,602,0
3,4885975,0,0.427024,-0.938137,0.215073,5,612,0
4,4700614,1,-1.979634,-1.048728,-2.321198,6,631,0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

col_target = "default"
col_features = df.columns.tolist()[1:-1] # Get all the columns except the customer_id and the target columns

X = np.array(df[col_features]).astype(float) # Convert the features to a numpy array
y = np.array(df[col_target]) # Convert the target to a numpy array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Créer le modèle de régression logistique
model = sm.Logit(y_train, X_train)

# Ajuster le modèle
result = model.fit()

# Obtenir le résumé du modèle
summary = result.summary()
print(summary)

# Faire des prédictions sur l'ensemble de test
y_pred_prob = result.predict(X_test)  # Probabilités prédites

# Convertir les probabilités en classes (0 ou 1) avec un seuil de 0.5
y_pred = (y_pred_prob > 0.5).astype(int)

# Évaluer le modèle
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 7000
Model:                          Logit   Df Residuals:                     6994
Method:                           MLE   Df Model:                            5
Date:                Tue, 06 Aug 2024   Pseudo R-squ.:                   1.000
Time:                        19:03:02   Log-Likelihood:            -0.00058575
converged:                      False   LL-Null:                       -3374.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2735.6947   1.26e+04      0.217      0.828    -2.2e+04    2.74e+04
x2           254.5035   1183.502      0.215      0.830   -2065.11

c:\Users\jcrig\Documents\FORMATION_SORBONNE\ML_Ops_Kamila_Kare\venv\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\jcrig\Documents\FORMATION_SORBONNE\ML_Ops_Kamila_Kare\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\jcrig\Documents\FORMATION_SORBONNE\ML_Ops_Kamila_Kare\venv\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


In [ ]:
from scipy.stats import pearsonr

